In [ ]:
#Run this in terminal
pip install tensorflow
pip install malaya
pip install PySastrawi

In [1]:
import numpy as np
import pandas as pd
import nltk
import string
import malaya
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras.utils import to_categorical
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from malaya import tokenizer
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

# Set random seed for reproducibility
np.random.seed(42)

Cannot import beam_search_ops from Tensorflow Addons, ['malaya.jawi_rumi.deep_model', 'malaya.phoneme.deep_model', 'malaya.rumi_jawi.deep_model', 'malaya.stem.deep_model'] will not available to use, make sure Tensorflow Addons version >= 0.12.0
check compatible Tensorflow version with Tensorflow Addons at https://github.com/tensorflow/addons/releases


In [116]:
# Define hyperparameters
vocab_size = 10000  # This is a placeholder value. Adjust based on actual vocabulary size.
embedding_dim = 50
max_length = 20  # This is a placeholder value. Adjust based on actual sentence length.
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<UNK>'
training_size = 16000  # This is a placeholder value. Adjust based on actual training data size.

In [31]:
#This one no need run le because i already run le 
import random

input_file = "train.txt"
output_file = "reduce-train.txt"
sample_percentage = 0.005

with open(input_file, 'r', encoding='utf-8') as infile:
    lines = infile.readlines()
    num_lines_to_keep = int(len(lines) * sample_percentage)
    selected_lines = random.sample(lines, num_lines_to_keep)

with open(output_file, 'w', encoding='utf-8') as outfile:
    outfile.writelines(selected_lines)

In [32]:
input_file = "valid.txt"
output_file = "reduce-valid.txt"
sample_percentage = 0.0009

with open(input_file, 'r', encoding='utf-8') as infile:
    lines = infile.readlines()
    num_lines_to_keep = int(len(lines) * sample_percentage)
    selected_lines = random.sample(lines, num_lines_to_keep)

with open(output_file, 'w', encoding='utf-8') as outfile:
    outfile.writelines(selected_lines)

In [33]:
input_file = "test.txt"
output_file = "reduce-test.txt"
sample_percentage = 0.0009

with open(input_file, 'r', encoding='utf-8') as infile:
    lines = infile.readlines()
    num_lines_to_keep = int(len(lines) * sample_percentage)
    selected_lines = random.sample(lines, num_lines_to_keep)

with open(output_file, 'w', encoding='utf-8') as outfile:
    outfile.writelines(selected_lines)

In [34]:
# Load data from text file
with open('reduce-train.txt', 'r', encoding='utf-8') as file:
    train = file.readlines()

with open('reduce-valid.txt', 'r', encoding='utf-8') as file:
    validate = file.readlines()

with open('reduce-test.txt', 'r', encoding='utf-8') as file:
    test = file.readlines()

# Display top lines
print(train[:10])

['Beruang mempunyai sistem penghadaman ringkas yang tipikal bagi karnivora, dengan sebiji perut, serangkai usus yang pendek dan tiada\xa0usus buntu. Panda gergasi yang maun pun kekal sistem penghadaman karnivor, serta gen-gen berkenaan karnivor. Kebolehannya untuk mencerna\xa0selulosa berpunca daripada kuman-kuman di dalam ususnya. Beruang mesti menghabiskan banyak masa dengan makan untuk mendapat cukup zat\xa0 daripada daun-daun. Apalagi si panda yang makan 12–15 jam sehari.\n', 'Buku Keadaan Dan Permasalahan Penulis Dan Penulisan Skrip merupakan sebuah buku yang diterbitkan oleh Finas dengan nombor ISBN 983-99423-0-1. \n', 'Macedonia Utara (), secara rasminya Republik Macedonia Utara () adalah sebuah negara terletak di Semenanjung Balkan, Eropah Tenggara. Ia merupakan salah satu negara pecahan kepada Republik Persekutuan Sosialis Yugoslavia yang merdeka pada tahun 1991. Antara tempoh ia dimerdekakan 1991 dan Februari 2019, ia dikenali secara rasminya sebagai Republik Macedonia , atau

In [117]:
#Perform text-preprocessing
nltk.download('punkt')
nltk.download('wordnet')
tokenizer = malaya.tokenizer.Tokenizer()


#Defini a function to covert
def load_custom_stopwords(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        stopwords_list = [line.strip() for line in file.readlines()]
    return set(stopwords_list)


#Load Malay Stopword
custom_stopwords_file = 'malay-stop-word.txt'
custom_stopwords = load_custom_stopwords(custom_stopwords_file)


def preprocess_text(text):
    # Step 1: Convert to lowercase
    text = text.lower()

    # Step 2: Remove Punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))

    # Step 3: Remove Numbers
    text = ''.join([i for i in text if not i.isdigit()])

    # Step 4: Remove Special Characters
    text = ''.join([i for i in text if i.isalnum() or i.isspace()])

    # Step 5: Remove Stop Words
    words = tokenizer.tokenize(text)
    text = ' '.join([word for word in words if word.lower() not in custom_stopwords])

    # Step 6: Lemmatization
    #naive = malaya.stem.naive()    
    # Lemmatize each token
    #text= [naive.stem(word) for word in text]

    

    return text

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [119]:
# Applying preprocessing to each document in the training set
preprocessed_train = [preprocess_text(text) for text in train]
# Applying preprocessing to each document in the validating set
preprocessed_validate = [preprocess_text(text) for text in validate]
# Applying preprocessing to each document in the testing set
preprocessed_test = [preprocess_text(text) for text in test]


In [120]:
preprocessed_train

['beruang sistem penghadaman ringkas tipikal karnivora sebiji perut serangkai usus pendek usus buntu panda gergasi maun kekal sistem penghadaman karnivor gengen karnivor kebolehannya mencerna selulosa berpunca kumankuman ususnya beruang mesti menghabiskan masa makan cukup zat daundaun apalagi si panda makan jam sehari',
 'buku keadaan permasalahan penulis penulisan skrip merupakan sebuah buku diterbitkan finas nombor isbn',
 'macedonia utara rasminya republik macedonia utara sebuah negara terletak semenanjung balkan eropah tenggara merupakan salah satu negara pecahan republik persekutuan sosialis yugoslavia merdeka tahun tempoh dimerdekakan februari dikenali rasminya republik macedonia pendeknya macedonia',
 'iklim basah taman nasional merupakan sumber mata air sungai alirannya kering sepanjang tahun delapan buah air terjun indah potensial kegiatan pariwisata alamrekreasi',
 'keseluruhannya konsert forteen membuktikan kumpulan sekadar menjual rupa sematamata sekali gus menaik taraf ker

In [121]:
# Tokenize sentences for 
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(preprocessed_train)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(preprocessed_train)
input_sequences = []
for sequence in sequences:
    for i in range(1, len(sequence)):
        n_gram_sequence = sequence[:i+1]
        input_sequences.append(n_gram_sequence)

input_sequences = pad_sequences(input_sequences, maxlen=max_length, padding='pre')
x_train, y_train = input_sequences[:,:-1], input_sequences[:,-1]
y_train = to_categorical(y_train, num_classes=vocab_size)

In [122]:
# Tokenize sentences for 
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(preprocessed_validate)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(preprocessed_validate)
input_sequences = []
for sequence in sequences:
    for i in range(1, len(sequence)):
        n_gram_sequence = sequence[:i+1]
        input_sequences.append(n_gram_sequence)

input_sequences = pad_sequences(input_sequences, maxlen=max_length, padding='pre')
x_valid, y_valid = input_sequences[:,:-1], input_sequences[:,-1]
y_valid = to_categorical(y_valid, num_classes=vocab_size)

In [123]:
# Tokenize sentences for 
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(preprocessed_validate)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(preprocessed_validate)
input_sequences = []
for sequence in sequences:
    for i in range(1, len(sequence)):
        n_gram_sequence = sequence[:i+1]
        input_sequences.append(n_gram_sequence)

input_sequences = pad_sequences(input_sequences, maxlen=max_length, padding='pre')
x_test, y_test = input_sequences[:,:-1], input_sequences[:,-1]
y_test = to_categorical(y_test, num_classes=vocab_size)

In [124]:
# Model definition
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length-1),
    LSTM(100),
    Dense(vocab_size, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 19, 50)            500000    
                                                                 
 lstm_5 (LSTM)               (None, 100)               60400     
                                                                 
 dense_5 (Dense)             (None, 10000)             1010000   
                                                                 
Total params: 1570400 (5.99 MB)
Trainable params: 1570400 (5.99 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [75]:
#Join all the vocab together
vocab= preprocessed_train + preprocessed_test + preprocessed_validate

# Set the size of the word vectors (embedding dimension)
vector_size = vocab_size
# Define and train the Word2Vec Skip-gram model
model = Word2Vec(sentences=[vocab], vector_size=vector_size, window=5, sg=1, min_count=1, workers=4)

# Save the trained model to a file
model.save("skipgram_word2vec.model")

# Access the word vectors
word_vectors = model.wv


In [126]:
# Train the model
model.fit(x_train, y_train, epochs=5, validation_data=(x_valid, y_valid), verbose=1)

Epoch 1/5
3011/3013 [============================>.] - ETA: 0s - loss: 7.3407 - accuracy: 0.1375WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x00000225E80A84A0> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: tensorflow_probability is not installed. Please install it and try again.
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: tensorflow_probability is not installed. Please install it and try again.
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
3013/3013 [==============================] - 31s 10ms/step - loss: 7.3405 - accur

In [127]:
loss, accuracy = model.evaluate(x_test, y_test, verbose=1)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

79/79 [==============================] - 0s 4ms/step - loss: 8.5113 - accuracy: 0.0028
Test Loss: 8.511252403259277
Test Accuracy: 0.0027866242453455925


In [131]:
def predict_next_word(model, tokenizer, text_sequence):
    sequence = tokenizer.texts_to_sequences([text_sequence])
    padded_sequence = pad_sequences(sequence, maxlen=max_length-1, padding='pre')
    prediction = model.predict(padded_sequence, verbose=0)
    predicted_word_index = np.argmax(prediction)
    predicted_word = tokenizer.index_word[predicted_word_index]
    return predicted_word

# Example usage
seed_text = "Saya "
predicted_word = predict_next_word(model, tokenizer, seed_text)
print(f'Given "{seed_text}", the next word might be "{predicted_word}".')

Given "Saya ", the next word might be "<UNK>".
